# Yaorui Guo(yg2815)
# Lab #6

In [1]:
#pip install linearmodels

In [49]:
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import linearmodels as lm
import seaborn as sns

from linearmodels import PanelOLS
from linearmodels import RandomEffects
from linearmodels import FirstDifferenceOLS

In [98]:
os.chdir('/Users/gyr/Desktop/5019_python/dataset')
g = pd.read_csv("panel-for-R.csv")
g.head()

,idnum,panelwave,ballot,form,formwt,oversamp,sample,panstat_2,panstat_3,mar1,...,wtpan12,wtpan123,wtpannr12,wtpannr123,xmarsex,xmovie,xnorcsiz,year,yearval,zodiac
0,9,1,3,2,1,1,9,1,1,5.0,...,0.414689,0.487828,0.435503,0.470575,1.0,2.0,1.0,2006.0,NaN,9.0
1,9,2,3,2,1,1,9,1,1,5.0,...,0.414689,0.487828,0.435503,0.470575,1.0,1.0,1.0,2008.0,NaN,9.0
2,9,3,3,2,1,1,9,1,1,1.0,...,0.414689,0.487828,0.435503,0.470575,1.0,1.0,1.0,2010.0,NaN,9.0
3,10,1,1,1,1,1,9,1,1,5.0,...,0.829377,0.858741,0.766632,0.828371,1.0,NaN,1.0,2006.0,NaN,3.0
4,10,2,1,1,1,1,9,1,1,5.0,...,0.829377,0.858741,0.766632,0.828371,1.0,NaN,1.0,2008.0,NaN,3.0


## The dependent variable I chose is 'refefam', which represents an opinion "It is much better for everyone involved if the man is the achiever outside the home and the woman takes care of the home and family", ranging from 1 to 4,  represents Strongly agree to Strongly disagree. to make the interpretation easier, I reverse code it. 

In [92]:
## reverse code variable refam- higher variable means people have Strongly agree opinion on this statement##
# g.fefam.value_counts()
g['rfefam'] = 5 - g.fefam

## I recoded the  variable 'marital' so that married represents a marriage status of 1 and all other statuses are treated as non-married

In [94]:
conditions = [
                (g['marital'] == 1),
                (g['marital'] != 1)]
choices = [1,0]
g['married'] = np.select(conditions, choices, default=np.nan)

In [95]:
g.married.value_counts()

0.0    3647
1.0    2353
Name: married, dtype: int64

# 1. Run a naive ("pooled") OLS regression on the panel data. Tell we how you expect your Xs to affect your Y and why. Interpret your results.

## I wonder if people who are married are more likely to agree that men are responsible for the home outside the home and women are responsible for taking care of the home

## I expect that before marriage people might have been more in denial about this statement, because everyone wants to be able to do everything and be on their own. After marriage, the burden of life gives everyone a clearer goal to help each other and understand each other in order to better maintain the marriage relationship. So there is a greater tendency to the traditional division of labor in the family, with the wage-earning for the men and unpaid maintenance of household and children for the women.

In [96]:
lm_ols = smf.ols(formula = 'rfefam ~ married ', data = g).fit()
print (lm_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 rfefam   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     18.34
Date:                Mon, 27 Jun 2022   Prob (F-statistic):           1.90e-05
Time:                        16:54:16   Log-Likelihood:                -3899.4
No. Observations:                3118   AIC:                             7803.
Df Residuals:                    3116   BIC:                             7815.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.1791      0.021    101.387      0.0

## From simple OLS result,  a married person scores 0.13 ponits higher on supporting the statement, meaning Married people tend to support this statement that men are responsible for the home outside the home and women are responsible for taking care of the home.

# 2. Run a first differences regression on the same model in Question 1. Interpret your results. Do you draw a different conclusion than in Question 1? Explain.

In [97]:
g = g.set_index(['idnum', 'panelwave'])
lm_fd = FirstDifferenceOLS.from_formula(formula = 'rfefam ~ married ', data = g)
res_fd = lm_fd.fit(cov_type='clustered', cluster_entity=True)
print(res_fd)

                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                 rfefam   R-squared:                     2.033e-05
Estimator:         FirstDifferenceOLS   R-squared (Between):             -0.0050
No. Observations:                1804   R-squared (Within):              -0.0001
Date:                Mon, Jun 27 2022   R-squared (Overall):             -0.0051
Time:                        16:54:24   Log-likelihood                   -2214.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.0366
Entities:                        1309   P-value                           0.8482
Avg Obs:                       2.3820   Distribution:                  F(1,1803)
Min Obs:                       1.0000                                           
Max Obs:                       3.0000   F-statistic (robust):             0.0308
                            

/Users/gyr/opt/anaconda3/lib/python3.9/site-packages/linearmodels/panel/model.py:2273: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


## From the first differences regression result, we can see there is no association between the change in a person's marital status from non-married to married and the attitude to this statement. The t-statistic is not statistical significant. This result is completely different from the conclusion in Q1. We cannot say that marriage makes one more inclined to support this statement because the relationship between these two variables is not statistically significant.

